In [ ]:
import pandas as pd

In [ ]:
# Step 1: Load the original dataset and the micro model predictions
original_df = pd.read_csv("enhanced_synthetic_discount_dataset.csv")
predicted_df = pd.read_csv("predicted_conversions.csv")

In [ ]:
# Step 2: Convert `event_date` to datetime format
original_df['event_date'] = pd.to_datetime(original_df['event_date'].astype(str), format='%Y-%m-%d')

In [ ]:
# Step 3: Keep only needed columns for merging
original_dates = original_df[['user_pseudo_id', 'item_id', 'event_date']]

In [ ]:
# Step 4: Merge event_date into predicted data
predicted_df = predicted_df.merge(original_dates, on=['user_pseudo_id', 'item_id'], how='left')

In [ ]:
# Step 5: Extract week from event_date (Monday-starting)
predicted_df['week'] = predicted_df['event_date'].dt.to_period('W').astype(str)

In [ ]:
# (Optional) Preview the result
print(predicted_df[['user_pseudo_id', 'item_id', 'event_date', 'week']].head())

  user_pseudo_id   item_id event_date                   week
0      user_1635  item_517 2021-01-13  2021-01-11/2021-01-17
1       user_912  item_509 2021-02-25  2021-02-22/2021-02-28
2       user_568  item_943 2021-01-18  2021-01-18/2021-01-24
3       user_947  item_896 2021-02-25  2021-02-22/2021-02-28
4       user_954  item_836 2021-01-17  2021-01-11/2021-01-17


Code: Weekly Discount-Bucket Aggregation

In [ ]:
# Step 1: Create discount buckets
bins = [0, 10, 20, 30, 40, 50, 60, 70, 100]
labels = ['0–10%', '10–20%', '20–30%', '30–40%', '40–50%', '50–60%', '60–70%', '70%+']
predicted_df['discount_bucket'] = pd.cut(predicted_df['discount_percent'], bins=bins, labels=labels, right=False)

In [ ]:
# Step 2: Calculate discounted price, revenue, and cost
predicted_df['discounted_price'] = predicted_df['original_price'] * (1 - predicted_df['discount_percent'] / 100)
predicted_df['expected_conversion'] = predicted_df['conversion_probability']
predicted_df['expected_revenue'] = predicted_df['expected_conversion'] * predicted_df['discounted_price']
predicted_df['expected_discount_cost'] = predicted_df['expected_conversion'] * (predicted_df['original_price'] - predicted_df['discounted_price'])

In [ ]:
# Step 3: Group by week and discount bucket
weekly_agg = predicted_df.groupby(['week', 'discount_bucket']).agg(
    expected_conversions=('expected_conversion', 'sum'),
    expected_revenue=('expected_revenue', 'sum'),
    expected_discount_cost=('expected_discount_cost', 'sum'),
    users_covered=('user_pseudo_id', 'nunique')
).reset_index()

/tmp/ipython-input-11-3875092722.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weekly_agg = predicted_df.groupby(['week', 'discount_bucket']).agg(


In [ ]:
# Step 4: Add CVR and Margin columns
weekly_agg['CVR'] = weekly_agg['expected_conversions'] / weekly_agg['users_covered']
weekly_agg['Margin'] = weekly_agg['expected_revenue'] - weekly_agg['expected_discount_cost']

In [ ]:
# Step 5: Save to file
weekly_agg.to_csv("macro_weekly_summary.csv", index=False)
print("✅ Weekly macro summary saved as 'macro_weekly_summary.csv'")

✅ Weekly macro summary saved as 'macro_weekly_summary.csv'
